---
Human Protein Atlas
---

https://www.proteinatlas.org

Download here: https://www.proteinatlas.org/about/download

In [1]:
import requests
import zipfile
import io
import os
import pandas as pd
import numpy as np
import urllib
import re
from io import StringIO

from toolbox import *

In [2]:
cfg = load_cfg()

logVersions = load_LogVersions()

# Download data

Can find versions there https://www.proteinatlas.org/about/download

In [3]:
def download_url(url, save_path_dir, oldFileName, newFileName, chunk_size=128):
    r = requests.get(url, stream=True)
    z = zipfile.ZipFile(io.BytesIO(r.content))
    z.extractall(save_path_dir)
    os.rename(os.path.join(save_path_dir, oldFileName), os.path.join(save_path_dir, newFileName))

In [4]:
versionHPA = '20-1'
versionEnsembl = '92-38'

**Normal tissue (from IHC)**

In [5]:
download_url(
    url='https://www.proteinatlas.org/download/normal_tissue.tsv.zip',
    save_path_dir=cfg['rawDataHPA'],
    oldFileName='normal_tissue.tsv',
    newFileName='normal_tissue_v{}--{}.tsv'.format(versionHPA,versionEnsembl),
)

**Aggregated "consensus" tissue transcriptomics (RNA-seq from HPA, GTEx, FANTOM5)**

In [6]:
download_url(
    url='https://www.proteinatlas.org/download/rna_tissue_consensus.tsv.zip',
    save_path_dir=cfg['rawDataHPA'],
    oldFileName='rna_consensus.tsv',
    newFileName='rna_tissue_consensus_v{}--{}.tsv'.format(versionHPA,versionEnsembl),
)

**Subcellular location (from IF)**

In [7]:
download_url(
    url='https://www.proteinatlas.org/download/subcellular_location.tsv.zip',
    save_path_dir=cfg['rawDataHPA'],
    oldFileName='subcellular_location.tsv',
    newFileName='subcellular_location_v{}--{}.tsv'.format(versionHPA,versionEnsembl),
)

In [8]:
# logVersions['HPA'] = dict()
logVersions['HPA']['rawData'] = {
        'HPA':versionHPA,
        'Ensembl':versionEnsembl
}

dump_LogVersions(logVersions)

**UniProt**

In [9]:
uniprotIDs = pd.read_csv(
        os.path.join(cfg['rawDataUniProt'], 
                     "uniprot_allProteins_Human_v{}.pkl".format(logVersions['UniProt']['rawData'])),
        header=None,
        names=['uniprotID']
    )
glance(uniprotIDs)

DataFrame: 20,386 rows 	 1 columns


,uniprotID
0,A0A024RBG1
1,A0A075B6H7
2,A0A075B6H8
3,A0A075B6H9
4,A0A075B6I0


## Mapping Ensembl IDs

In [10]:
tissueIHC = pd.read_csv(
    os.path.join(cfg['rawDataHPA'], 'normal_tissue_v{}--{}.tsv'.format(versionHPA,versionEnsembl)),
    sep = "\t"
)
glance(tissueIHC)

DataFrame: 1,118,517 rows 	 6 columns


,Gene,Gene name,Tissue,Cell type,Level,Reliability
0,ENSG00000000003,TSPAN6,adipose tissue,adipocytes,Not detected,Approved
1,ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected,Approved
2,ENSG00000000003,TSPAN6,appendix,glandular cells,Medium,Approved
3,ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected,Approved
4,ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected,Approved


In [11]:
consensusRNAseq = pd.read_csv(
    os.path.join(cfg['rawDataHPA'], 'rna_tissue_consensus_v{}--{}.tsv'.format(versionHPA,versionEnsembl)),
    sep = "\t"
)
glance(consensusRNAseq)

DataFrame: 1,180,464 rows 	 4 columns


,Gene,Gene name,Tissue,NX
0,ENSG00000000003,TSPAN6,adipose tissue,27.0
1,ENSG00000000003,TSPAN6,adrenal gland,9.8
2,ENSG00000000003,TSPAN6,amygdala,7.0
3,ENSG00000000003,TSPAN6,appendix,4.4
4,ENSG00000000003,TSPAN6,B-cells,0.3


In [12]:
subcellularLocation = pd.read_csv(
    os.path.join(cfg['rawDataHPA'], 'subcellular_location_v{}--{}.tsv'.format(versionHPA,versionEnsembl)),
    sep = "\t"
)
glance(subcellularLocation)

DataFrame: 12,813 rows 	 14 columns


,Gene,Gene name,Reliability,Main location,Additional location,Extracellular location,Enhanced,Supported,Approved,Uncertain,Single-cell variation intensity,Single-cell variation spatial,Cell cycle dependency,GO id
0,ENSG00000000003,TSPAN6,Approved,Cell Junctions;Cytosol,Nucleoli fibrillar center,NaN,NaN,NaN,Cell Junctions;Cytosol;Nucleoli fibrillar center,NaN,Cytosol,NaN,NaN,Cell Junctions (GO:0030054);Cytosol (GO:000582...
1,ENSG00000000457,SCYL3,Uncertain,Microtubules,Nuclear bodies,NaN,NaN,NaN,NaN,Microtubules;Nuclear bodies,NaN,NaN,NaN,Microtubules (GO:0015630);Nuclear bodies (GO:0...
2,ENSG00000000460,C1orf112,Approved,Mitochondria,NaN,NaN,NaN,NaN,Mitochondria,NaN,NaN,NaN,NaN,Mitochondria (GO:0005739)
3,ENSG00000000938,FGR,Approved,Plasma membrane,Aggresome,NaN,NaN,NaN,Aggresome;Plasma membrane,NaN,NaN,NaN,NaN,Aggresome (GO:0016235);Plasma membrane (GO:000...
4,ENSG00000000971,CFH,Approved,Vesicles,NaN,Predicted to be secreted,NaN,NaN,Vesicles,NaN,NaN,NaN,NaN,Vesicles (GO:0043231)


In [13]:
geneList = list(set(tissueIHC['Gene'])) + list(set(consensusRNAseq['Gene'])) + list(set(subcellularLocation['Gene']))
# geneList = list(set(subcellularLocation['Gene']))
geneList2 = list(set(geneList))
print(len(geneList2))

19670


In [14]:
HPAmapping0 = mappingUniprotIDs(fromID = 'ENSEMBL_ID', listIDs = geneList2)
HPAmapping0 = HPAmapping0.rename(columns={
    'From': 'Gene', 
    'To': 'uniprotID'
})

glance(HPAmapping0)

DataFrame: 74,614 rows 	 2 columns


,Gene,uniprotID
0,ENSG00000004700,P46063
1,ENSG00000004700,F5GYB7
2,ENSG00000004700,F5H2L2
3,ENSG00000004700,F5H3W0
4,ENSG00000004700,F5H4P4


In [15]:
print("{:,}/{:,} couldn't be matched".format(len(geneList2)-len(set(HPAmapping0.Gene)), len(geneList2)))

207/19,670 couldn't be matched


**Remove unreviewed proteins**

In [16]:
HPAmapping = HPAmapping0.merge(uniprotIDs, how='inner', on='uniprotID')
glance(HPAmapping)

DataFrame: 19,049 rows 	 2 columns


,Gene,uniprotID
0,ENSG00000004700,P46063
1,ENSG00000153132,O14967
2,ENSG00000135452,Q12999
3,ENSG00000182919,Q9H0W9
4,ENSG00000180785,Q8TCB6


In [17]:
print(len(set(HPAmapping0.Gene)))
print(len(set(HPAmapping0.uniprotID)))

19463
74406


In [18]:
print(len(set(HPAmapping.Gene)))
print(len(set(HPAmapping.uniprotID)))

19018
18938


---
**Look at genes matching multiple proteins**

In [19]:
HPAmapping.loc[HPAmapping.duplicated(subset=["Gene"], keep=False)].sort_values(by=["Gene"])

,Gene,uniprotID
10898,ENSG00000021645,Q9HDB5
10899,ENSG00000021645,Q9Y4C0
18409,ENSG00000079393,Q9UII6
18408,ENSG00000079393,Q6B8I1
18004,ENSG00000087460,P63092
18006,ENSG00000087460,Q5JWF2
18005,ENSG00000087460,P84996
18003,ENSG00000087460,O95467
13405,ENSG00000100300,B1AH88
13406,ENSG00000100300,P30536


---
**Export**

In [20]:
HPAmapping.to_pickle(os.path.join(cfg['rawDataHPA'],'GeneMappingHPA__v{}--{}.pkl'.format(versionHPA,versionEnsembl)))

# Preprocessing

In [21]:
logVersions = load_LogVersions()

In [22]:
HPAmapping = pd.read_pickle(os.path.join(cfg['rawDataHPA'],'GeneMappingHPA__v{}--{}.pkl'.format(logVersions['HPA']['rawData']['HPA'],logVersions['HPA']['rawData']['Ensembl'])))


## Tissue IHC data

In [23]:
tissueIHC = pd.read_csv(
    os.path.join(cfg['rawDataHPA'], 'normal_tissue_v{}--{}.tsv'.format(logVersions['HPA']['rawData']['HPA'],logVersions['HPA']['rawData']['Ensembl'])),
    sep = "\t"
)
glance(tissueIHC)

DataFrame: 1,118,517 rows 	 6 columns


,Gene,Gene name,Tissue,Cell type,Level,Reliability
0,ENSG00000000003,TSPAN6,adipose tissue,adipocytes,Not detected,Approved
1,ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected,Approved
2,ENSG00000000003,TSPAN6,appendix,glandular cells,Medium,Approved
3,ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected,Approved
4,ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected,Approved


---
**EDA**

In [24]:
print(tissueIHC.loc[:,"Reliability"].value_counts())
print()
print(tissueIHC.loc[:,"Level"].value_counts())

Approved     444411
Enhanced     344216
Uncertain    188934
Supported    140956
Name: Reliability, dtype: int64

Not detected          516415
Medium                293549
Low                   179696
High                  126901
Not representative        25
Name: Level, dtype: int64


In [25]:
print('Number of genes: {:,}'.format(len(set(tissueIHC.Gene))))
print('Number of tissues: {:,}'.format(len(set(tissueIHC.Tissue))))
print('Number of cell types: {:,}'.format(len(set(tissueIHC['Cell type']))))

Number of genes: 15,320
Number of tissues: 63
Number of cell types: 120


In [26]:
print(tissueIHC.Tissue.value_counts())
print()
print(tissueIHC['Cell type'].value_counts())

testis               57064
cerebellum           54273
cerebral cortex      53644
skin 1               52915
breast               39598
tonsil               39594
colon                37502
soft tissue 1        28518
kidney               27052
hippocampus          26791
lung                 26754
soft tissue 2        26753
endometrium 1        26746
endometrium 2        26702
liver                26691
spleen               26675
caudate              26610
pancreas             26182
appendix             26087
lymph node           25914
placenta             25329
cervix, uterine      25045
ovary                21024
bronchus             18916
fallopian tube       18336
nasopharynx          18231
adrenal gland        13604
duodenum             13387
heart muscle         13387
smooth muscle        13386
                     ...  
skeletal muscle      13379
salivary gland       13370
stomach 1            13351
stomach 2            13349
esophagus            13345
prostate             13340
r

---
**Select high quality calls**

Here, we keep `enhanced`, `supported` and `approved` calls, we discard `uncertain`.

In [27]:
tissueIHC2 = tissueIHC.loc[tissueIHC.Reliability != "Uncertain"].drop("Reliability", axis=1)

glance(tissueIHC2)

DataFrame: 929,583 rows 	 5 columns


,Gene,Gene name,Tissue,Cell type,Level
0,ENSG00000000003,TSPAN6,adipose tissue,adipocytes,Not detected
1,ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected
2,ENSG00000000003,TSPAN6,appendix,glandular cells,Medium
3,ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected
4,ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected


In [28]:
print('Number of genes: {:,}'.format(len(set(tissueIHC2.Gene))))
print('...{:,} genes lost'.format(len(set(tissueIHC.Gene))-len(set(tissueIHC2.Gene))))

Number of genes: 11,006
...4,314 genes lost


---
**Create quantitative variable**

In [29]:
def mappingExpression(x):
    if x == 'High':
        return 3
    elif x == 'Medium':
        return 2
    elif x == 'Low':
        return 1
    else:
        return -1

tissueIHC2['ExpressionQuant'] = tissueIHC2.Level.apply(mappingExpression)

glance(tissueIHC2, n=20)

DataFrame: 929,583 rows 	 6 columns


,Gene,Gene name,Tissue,Cell type,Level,ExpressionQuant
0,ENSG00000000003,TSPAN6,adipose tissue,adipocytes,Not detected,-1
1,ENSG00000000003,TSPAN6,adrenal gland,glandular cells,Not detected,-1
2,ENSG00000000003,TSPAN6,appendix,glandular cells,Medium,2
3,ENSG00000000003,TSPAN6,appendix,lymphoid tissue,Not detected,-1
4,ENSG00000000003,TSPAN6,bone marrow,hematopoietic cells,Not detected,-1
5,ENSG00000000003,TSPAN6,breast,adipocytes,Not detected,-1
6,ENSG00000000003,TSPAN6,breast,glandular cells,High,3
7,ENSG00000000003,TSPAN6,breast,myoepithelial cells,Not detected,-1
8,ENSG00000000003,TSPAN6,bronchus,respiratory epithelial cells,High,3
9,ENSG00000000003,TSPAN6,caudate,glial cells,Not detected,-1


### Pivot table using (tissue;cell type) as keys

In [30]:
# Sanity check
assert ~any(tissueIHC2[['Gene', 'Tissue', 'Cell type']].duplicated())

In [31]:
tissueIHC_pivot1 = pd.pivot_table(
    data = tissueIHC2,
    index='Gene', 
    columns = ['Tissue', 'Cell type'],
    values = 'ExpressionQuant'
)

tissueIHC_pivot1 = tissueIHC_pivot1.reset_index()

glance(tissueIHC_pivot1)

tissueIHC_pivot1.columns = [' '.join(col).strip() for col in tissueIHC_pivot1.columns.values]

glance(tissueIHC_pivot1)

DataFrame: 11,006 rows 	 190 columns


Tissue                Gene adipose tissue             adrenal gland  \
Cell type                      adipocytes cells in zona fasciculata   
0          ENSG00000000003           -1.0                       NaN   
1          ENSG00000000419            2.0                       NaN   
2          ENSG00000000457            1.0                       NaN   
3          ENSG00000000938           -1.0                       NaN   
4          ENSG00000000971           -1.0                       NaN   

Tissue                                                                         \
Cell type cells in zona glomerulosa cells in zona reticularis glandular cells   
0                               NaN                       NaN            -1.0   
1                               NaN                       NaN             3.0   
2                               NaN                       NaN            -1.0   
3                               NaN                       NaN            -1.0   
4                               NaN                       NaN            -1.0   

Tissue                           appendix                         bone marrow  \
Cell type medullary cells glandular cells lymphoid tissue hematopoietic cells   
0                     NaN             2.0            -1.0                -1.0   
1                     NaN             3.0             2.0                 2.0   
2                     NaN             1.0            -1.0                 1.0   
3                     NaN            -1.0             NaN                 2.0   
4                     NaN            -1.0            -1.0                -1.0   

Tissue     ...        testis                             thymus  \
Cell type  ... sertoli cells spermatogonia cells cortical cells   
0          ...           NaN                 NaN            NaN   
1          ...           NaN                 NaN            NaN   
2          ...           1.0                 1.0            NaN   
3          ...           NaN                 NaN            NaN   
4          ...          -1.0                -1.0            NaN   

Tissue                      thyroid gland                tonsil  \
Cell type medullary cells glandular cells germinal center cells   
0                     NaN             2.0                  -1.0   
1                     NaN             2.0                   1.0   
2                     NaN             2.0                   2.0   
3                     NaN            -1.0                   2.0   
4                     NaN            -1.0                  -1.0   

Tissue                                                         \
Cell type non-germinal center cells squamous epithelial cells   
0                              -1.0                       3.0   
1                               2.0                       2.0   
2                               1.0                       2.0   
3                               2.0                       1.0   
4                              -1.0                      -1.0   

Tissue     urinary bladder                    vagina  
Cell type urothelial cells squamous epithelial cells  
0                      3.0                       3.0  
1                      2.0                       2.0  
2                      3.0                       1.0  
3                     -1.0                      -1.0  
4                     -1.0                      -1.0  

[5 rows x 190 columns]

DataFrame: 11,006 rows 	 190 columns


,Gene,adipose tissue adipocytes,adrenal gland cells in zona fasciculata,adrenal gland cells in zona glomerulosa,adrenal gland cells in zona reticularis,adrenal gland glandular cells,adrenal gland medullary cells,appendix glandular cells,appendix lymphoid tissue,bone marrow hematopoietic cells,...,testis sertoli cells,testis spermatogonia cells,thymus cortical cells,thymus medullary cells,thyroid gland glandular cells,tonsil germinal center cells,tonsil non-germinal center cells,tonsil squamous epithelial cells,urinary bladder urothelial cells,vagina squamous epithelial cells
0,ENSG00000000003,-1.0,NaN,NaN,NaN,-1.0,NaN,2.0,-1.0,-1.0,...,NaN,NaN,NaN,NaN,2.0,-1.0,-1.0,3.0,3.0,3.0
1,ENSG00000000419,2.0,NaN,NaN,NaN,3.0,NaN,3.0,2.0,2.0,...,NaN,NaN,NaN,NaN,2.0,1.0,2.0,2.0,2.0,2.0
2,ENSG00000000457,1.0,NaN,NaN,NaN,-1.0,NaN,1.0,-1.0,1.0,...,1.0,1.0,NaN,NaN,2.0,2.0,1.0,2.0,3.0,1.0
3,ENSG00000000938,-1.0,NaN,NaN,NaN,-1.0,NaN,-1.0,NaN,2.0,...,NaN,NaN,NaN,NaN,-1.0,2.0,2.0,1.0,-1.0,-1.0
4,ENSG00000000971,-1.0,NaN,NaN,NaN,-1.0,NaN,-1.0,-1.0,-1.0,...,-1.0,-1.0,NaN,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


In [32]:
# Sanity check
# Check that there is the same frquency of each level of expression before and after the pivot.
foo = tissueIHC_pivot1.set_index(["Gene"]).stack().value_counts()
bar = tissueIHC2['ExpressionQuant'].value_counts()
assert (foo == bar).all()

---
**EDA**

In [33]:
NAperGene = tissueIHC_pivot1.isnull().sum(axis=1).astype(int)

print("Missing values per gene")
print('------')
print("mean: {:.2f} \t median: {:.2f} \t min: {:.0f} \t max: {:.0f}".format(np.mean(NAperGene),
                                                                            np.median(
                                                                                NAperGene),
                                                                            np.min(
                                                                                NAperGene),
                                                                            np.max(NAperGene)))
print("(out of {} tissues)".format(tissueIHC_pivot1.shape[1]))

Missing values per gene
------
mean: 104.54 	 median: 106.00 	 min: 65 	 max: 121
(out of 190 tissues)


___
**Match Ensembl IDs with UniProt IDs**

In [34]:
tissueIHC_pivot1_2 = tissueIHC_pivot1.merge(HPAmapping, how="inner", on = "Gene")

glance(tissueIHC_pivot1_2)

DataFrame: 10,921 rows 	 191 columns


,Gene,adipose tissue adipocytes,adrenal gland cells in zona fasciculata,adrenal gland cells in zona glomerulosa,adrenal gland cells in zona reticularis,adrenal gland glandular cells,adrenal gland medullary cells,appendix glandular cells,appendix lymphoid tissue,bone marrow hematopoietic cells,...,testis spermatogonia cells,thymus cortical cells,thymus medullary cells,thyroid gland glandular cells,tonsil germinal center cells,tonsil non-germinal center cells,tonsil squamous epithelial cells,urinary bladder urothelial cells,vagina squamous epithelial cells,uniprotID
0,ENSG00000000003,-1.0,NaN,NaN,NaN,-1.0,NaN,2.0,-1.0,-1.0,...,NaN,NaN,NaN,2.0,-1.0,-1.0,3.0,3.0,3.0,O43657
1,ENSG00000000419,2.0,NaN,NaN,NaN,3.0,NaN,3.0,2.0,2.0,...,NaN,NaN,NaN,2.0,1.0,2.0,2.0,2.0,2.0,O60762
2,ENSG00000000457,1.0,NaN,NaN,NaN,-1.0,NaN,1.0,-1.0,1.0,...,1.0,NaN,NaN,2.0,2.0,1.0,2.0,3.0,1.0,Q8IZE3
3,ENSG00000000938,-1.0,NaN,NaN,NaN,-1.0,NaN,-1.0,NaN,2.0,...,NaN,NaN,NaN,-1.0,2.0,2.0,1.0,-1.0,-1.0,P09769
4,ENSG00000000971,-1.0,NaN,NaN,NaN,-1.0,NaN,-1.0,-1.0,-1.0,...,-1.0,NaN,NaN,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,P08603


In [35]:
# Sanity check
assert tissueIHC_pivot1_2.uniprotID.isna().sum() == 0

*Deal with proteins matching several genes*

In [36]:
if len(set(tissueIHC_pivot1_2.uniprotID)) != len(tissueIHC_pivot1_2):
    tissueIHC_pivot1_2 = tissueIHC_pivot1_2.groupby('uniprotID').max().reset_index()
    print('New number of proteins: {:,}'.format(len(tissueIHC_pivot1_2)))

New number of proteins: 10,850


In [37]:
# Sanity check
assert len(set(tissueIHC_pivot1_2.uniprotID)) == len(tissueIHC_pivot1_2)

---
**Fill missing values with 0**

In [38]:
tissueIHC_pivot1_3 = tissueIHC_pivot1_2.fillna(0)
glance(tissueIHC_pivot1_3)

DataFrame: 10,850 rows 	 191 columns


,uniprotID,Gene,adipose tissue adipocytes,adrenal gland cells in zona fasciculata,adrenal gland cells in zona glomerulosa,adrenal gland cells in zona reticularis,adrenal gland glandular cells,adrenal gland medullary cells,appendix glandular cells,appendix lymphoid tissue,...,testis sertoli cells,testis spermatogonia cells,thymus cortical cells,thymus medullary cells,thyroid gland glandular cells,tonsil germinal center cells,tonsil non-germinal center cells,tonsil squamous epithelial cells,urinary bladder urothelial cells,vagina squamous epithelial cells
0,A0A024RBG1,ENSG00000177144,2.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,...,0.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,1.0
1,A0A087WVF3,ENSG00000274419,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,...,1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
2,A0A087WXS9,ENSG00000274933,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,...,1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,A0A087X179,ENSG00000278599,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,...,1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,A0A087X1G2,ENSG00000273513,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,...,1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


---
**Remove gene ID**

In [39]:
tissueIHC_pivot1_4 = tissueIHC_pivot1_3.drop(['Gene'], axis=1)
glance(tissueIHC_pivot1_4)

DataFrame: 10,850 rows 	 190 columns


,uniprotID,adipose tissue adipocytes,adrenal gland cells in zona fasciculata,adrenal gland cells in zona glomerulosa,adrenal gland cells in zona reticularis,adrenal gland glandular cells,adrenal gland medullary cells,appendix glandular cells,appendix lymphoid tissue,bone marrow hematopoietic cells,...,testis sertoli cells,testis spermatogonia cells,thymus cortical cells,thymus medullary cells,thyroid gland glandular cells,tonsil germinal center cells,tonsil non-germinal center cells,tonsil squamous epithelial cells,urinary bladder urothelial cells,vagina squamous epithelial cells
0,A0A024RBG1,2.0,0.0,0.0,0.0,2.0,0.0,2.0,2.0,1.0,...,0.0,0.0,0.0,0.0,2.0,2.0,2.0,1.0,2.0,1.0
1,A0A087WVF3,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,...,1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,1.0,-1.0
2,A0A087WXS9,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,...,1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
3,A0A087X179,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,...,1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0
4,A0A087X1G2,-1.0,0.0,0.0,0.0,-1.0,0.0,-1.0,-1.0,-1.0,...,1.0,-1.0,0.0,0.0,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0


---
**Export**

- v2.0: uses HPA v19.3 and Ensembl v92.38
- v2.1: uses HPA v20.1 and Ensembl v92.38

In [40]:
tissueIHC_pivot1_export = tissueIHC_pivot1_4

In [41]:
myVersionHPA = '2-1'

logVersions['HPA']['preprocessed']['tissueIHC_tissueCell'] = myVersionHPA

dump_LogVersions(logVersions)

In [42]:
tissueIHC_pivot1_export.to_pickle(os.path.join(cfg['outputPreprocessingHPA'], "tissueIHC_tissueCell_v{}.pkl".format(myVersionHPA)))


### Pivot table using (tissue) only as key

**Group by tissue**

In [43]:
tissueIHC_groupedTissue = tissueIHC2.loc[:,['Gene','Tissue','ExpressionQuant']].groupby(['Gene','Tissue']).max().reset_index()
glance(tissueIHC_groupedTissue)

DataFrame: 536,184 rows 	 3 columns


,Gene,Tissue,ExpressionQuant
0,ENSG00000000003,adipose tissue,-1
1,ENSG00000000003,adrenal gland,-1
2,ENSG00000000003,appendix,2
3,ENSG00000000003,bone marrow,-1
4,ENSG00000000003,breast,3


**Pivot**

In [44]:
# Sanity check
assert ~any(tissueIHC_groupedTissue[['Gene', 'Tissue']].duplicated())

In [45]:
tissueIHC_pivot2 = pd.pivot_table(
    data = tissueIHC_groupedTissue,
    index='Gene', 
    columns = ['Tissue'],
    values = 'ExpressionQuant'
).reset_index()

glance(tissueIHC_pivot2)

DataFrame: 11,006 rows 	 63 columns


Tissue,Gene,adipose tissue,adrenal gland,appendix,bone marrow,breast,bronchus,cartilage,caudate,cerebellum,...,spleen,stomach 1,stomach 2,substantia nigra,testis,thymus,thyroid gland,tonsil,urinary bladder,vagina
0,ENSG00000000003,-1.0,-1.0,2.0,-1.0,3.0,3.0,NaN,-1.0,-1.0,...,-1.0,2.0,2.0,NaN,1.0,NaN,2.0,3.0,3.0,3.0
1,ENSG00000000419,2.0,3.0,3.0,2.0,2.0,2.0,NaN,2.0,2.0,...,2.0,3.0,3.0,NaN,3.0,NaN,2.0,2.0,2.0,2.0
2,ENSG00000000457,1.0,-1.0,1.0,1.0,2.0,-1.0,NaN,-1.0,1.0,...,1.0,2.0,2.0,NaN,2.0,NaN,2.0,2.0,3.0,1.0
3,ENSG00000000938,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,NaN,-1.0,-1.0,...,2.0,-1.0,-1.0,NaN,-1.0,NaN,-1.0,2.0,-1.0,-1.0
4,ENSG00000000971,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,-1.0,-1.0,...,-1.0,-1.0,-1.0,NaN,3.0,NaN,-1.0,-1.0,-1.0,-1.0


---
**EDA**

In [46]:
NAperGene = tissueIHC_pivot2.isnull().sum(axis=1).astype(int)

print("Missing values per gene")
print('------')
print("mean: {:.2f} \t median: {:.2f} \t min: {:.0f} \t max: {:.0f}".format(np.mean(NAperGene),
                                                                            np.median(
                                                                                NAperGene),
                                                                            np.min(
                                                                                NAperGene),
                                                                            np.max(NAperGene)))
print("(out of {} tissues)".format(tissueIHC_pivot2.shape[1]))

Missing values per gene
------
mean: 13.28 	 median: 13.00 	 min: 11 	 max: 18
(out of 63 tissues)


___
**Match Ensembl IDs with UniProt IDs**

In [47]:
tissueIHC_pivot2_2 = tissueIHC_pivot2.merge(HPAmapping, how="inner", on = "Gene")

glance(tissueIHC_pivot2_2)

DataFrame: 10,921 rows 	 64 columns


,Gene,adipose tissue,adrenal gland,appendix,bone marrow,breast,bronchus,cartilage,caudate,cerebellum,...,stomach 1,stomach 2,substantia nigra,testis,thymus,thyroid gland,tonsil,urinary bladder,vagina,uniprotID
0,ENSG00000000003,-1.0,-1.0,2.0,-1.0,3.0,3.0,NaN,-1.0,-1.0,...,2.0,2.0,NaN,1.0,NaN,2.0,3.0,3.0,3.0,O43657
1,ENSG00000000419,2.0,3.0,3.0,2.0,2.0,2.0,NaN,2.0,2.0,...,3.0,3.0,NaN,3.0,NaN,2.0,2.0,2.0,2.0,O60762
2,ENSG00000000457,1.0,-1.0,1.0,1.0,2.0,-1.0,NaN,-1.0,1.0,...,2.0,2.0,NaN,2.0,NaN,2.0,2.0,3.0,1.0,Q8IZE3
3,ENSG00000000938,-1.0,-1.0,-1.0,2.0,-1.0,-1.0,NaN,-1.0,-1.0,...,-1.0,-1.0,NaN,-1.0,NaN,-1.0,2.0,-1.0,-1.0,P09769
4,ENSG00000000971,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,NaN,-1.0,-1.0,...,-1.0,-1.0,NaN,3.0,NaN,-1.0,-1.0,-1.0,-1.0,P08603


In [48]:
# Sanity check
assert tissueIHC_pivot2_2.uniprotID.isna().sum() == 0

*Deal with proteins matching several genes*

In [49]:
if len(set(tissueIHC_pivot2_2.uniprotID)) != len(tissueIHC_pivot2_2):
    tissueIHC_pivot2_2 = tissueIHC_pivot2_2.groupby('uniprotID').max().reset_index()
    print('New number of proteins: {:,}'.format(len(tissueIHC_pivot2_2)))

New number of proteins: 10,850


In [50]:
# Sanity check
assert len(set(tissueIHC_pivot2_2.uniprotID)) == len(tissueIHC_pivot2_2)

---
**Fill missing values with 0**

In [51]:
tissueIHC_pivot2_3 = tissueIHC_pivot2_2.fillna(0)
glance(tissueIHC_pivot2_3)

DataFrame: 10,850 rows 	 64 columns


,uniprotID,Gene,adipose tissue,adrenal gland,appendix,bone marrow,breast,bronchus,cartilage,caudate,...,spleen,stomach 1,stomach 2,substantia nigra,testis,thymus,thyroid gland,tonsil,urinary bladder,vagina
0,A0A024RBG1,ENSG00000177144,2.0,2.0,2.0,1.0,2.0,2.0,0.0,2.0,...,1.0,1.0,2.0,0.0,2.0,0.0,2.0,2.0,2.0,1.0
1,A0A087WVF3,ENSG00000274419,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,...,-1.0,-1.0,-1.0,0.0,3.0,0.0,-1.0,-1.0,1.0,-1.0
2,A0A087WXS9,ENSG00000274933,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,...,-1.0,-1.0,-1.0,0.0,3.0,0.0,-1.0,-1.0,-1.0,-1.0
3,A0A087X179,ENSG00000278599,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,...,-1.0,-1.0,-1.0,0.0,3.0,0.0,-1.0,-1.0,-1.0,-1.0
4,A0A087X1G2,ENSG00000273513,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,...,-1.0,-1.0,-1.0,0.0,3.0,0.0,-1.0,-1.0,-1.0,-1.0


---
**Remove gene ID**

In [52]:
tissueIHC_pivot2_4 = tissueIHC_pivot2_3.drop(['Gene'], axis=1)
glance(tissueIHC_pivot2_4)

DataFrame: 10,850 rows 	 63 columns


,uniprotID,adipose tissue,adrenal gland,appendix,bone marrow,breast,bronchus,cartilage,caudate,cerebellum,...,spleen,stomach 1,stomach 2,substantia nigra,testis,thymus,thyroid gland,tonsil,urinary bladder,vagina
0,A0A024RBG1,2.0,2.0,2.0,1.0,2.0,2.0,0.0,2.0,2.0,...,1.0,1.0,2.0,0.0,2.0,0.0,2.0,2.0,2.0,1.0
1,A0A087WVF3,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,0.0,3.0,0.0,-1.0,-1.0,1.0,-1.0
2,A0A087WXS9,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,0.0,3.0,0.0,-1.0,-1.0,-1.0,-1.0
3,A0A087X179,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,0.0,3.0,0.0,-1.0,-1.0,-1.0,-1.0
4,A0A087X1G2,-1.0,-1.0,-1.0,-1.0,-1.0,-1.0,0.0,-1.0,-1.0,...,-1.0,-1.0,-1.0,0.0,3.0,0.0,-1.0,-1.0,-1.0,-1.0


---
**Export**

- v2.0: uses HPA v19.3 and Ensembl v92.38
- v2.1: uses HPA v20.1 and Ensembl v92.38

In [53]:
tissueIHC_pivot2_export = tissueIHC_pivot2_4

In [54]:
myVersionHPA = '2-1'

logVersions['HPA']['preprocessed']['tissueIHC_tissueOnly'] = myVersionHPA

dump_LogVersions(logVersions)

In [55]:
tissueIHC_pivot2_export.to_pickle(os.path.join(cfg['outputPreprocessingHPA'], "tissueIHC_tissueOnly_v{}.pkl".format(myVersionHPA)))


## Concensus RNA-seq data

In [56]:
consensusRNAseq = pd.read_csv(
    os.path.join(cfg['rawDataHPA'], 'rna_tissue_consensus_v{}--{}.tsv'.format(logVersions['HPA']['rawData']['HPA'],logVersions['HPA']['rawData']['Ensembl'])),
    sep = "\t"
)
glance(consensusRNAseq)

DataFrame: 1,180,464 rows 	 4 columns


,Gene,Gene name,Tissue,NX
0,ENSG00000000003,TSPAN6,adipose tissue,27.0
1,ENSG00000000003,TSPAN6,adrenal gland,9.8
2,ENSG00000000003,TSPAN6,amygdala,7.0
3,ENSG00000000003,TSPAN6,appendix,4.4
4,ENSG00000000003,TSPAN6,B-cells,0.3


---
**EDA**

In [57]:
print('Number of genes: {:,}'.format(len(set(consensusRNAseq.Gene))))
print('Number of tissues: {:,}'.format(len(set(consensusRNAseq.Tissue))))

Number of genes: 19,670
Number of tissues: 61


In [58]:
consensusRNAseq.NX.describe()

count    1.180464e+06
mean     1.115004e+01
std      2.703313e+01
min      0.000000e+00
25%      7.000000e-01
50%      5.700000e+00
75%      1.480000e+01
max      6.520000e+03
Name: NX, dtype: float64

---
**Pivot**

In [59]:
# Sanity check
assert ~any(consensusRNAseq[['Gene', 'Tissue']].duplicated())

In [60]:
consensusRNAseq_pivot = pd.pivot_table(
    data = consensusRNAseq,
    index='Gene', 
    columns = ['Tissue'],
    values = 'NX'
).reset_index()

glance(consensusRNAseq_pivot)

DataFrame: 19,670 rows 	 62 columns


Tissue,Gene,B-cells,NK-cells,T-cells,adipose tissue,adrenal gland,amygdala,appendix,basal ganglia,bone marrow,...,stomach,testis,thalamus,thymus,thyroid gland,tongue,tonsil,total PBMC,urinary bladder,vagina
0,ENSG00000000003,0.3,0.3,6.6,27.0,9.8,7.0,4.4,6.6,0.1,...,8.0,17.5,6.6,2.9,24.6,11.0,19.8,1.4,17.2,11.5
1,ENSG00000000005,0.0,0.0,0.0,18.4,0.1,0.1,0.4,0.0,0.0,...,0.1,0.2,0.0,0.2,0.1,2.6,0.4,0.0,0.2,0.1
2,ENSG00000000419,20.7,20.3,25.1,32.3,39.7,20.2,23.3,30.0,32.9,...,30.4,25.6,21.7,36.1,34.5,49.1,36.4,15.7,35.3,28.3
3,ENSG00000000457,8.3,11.8,10.5,10.8,11.4,8.5,15.4,10.6,5.0,...,11.3,9.1,8.9,17.2,13.3,9.5,16.0,5.9,16.0,12.7
4,ENSG00000000460,2.9,4.9,1.6,3.4,3.1,3.3,7.4,4.9,10.1,...,3.0,19.7,5.9,24.6,3.2,2.9,9.5,1.6,4.0,3.3


---
**EDA**

In [61]:
NAperGene = consensusRNAseq_pivot.isnull().sum(axis=1).astype(int)

print("Missing values per gene")
print('------')
print("mean: {:.2f} \t median: {:.2f} \t min: {:.0f} \t max: {:.0f}".format(np.mean(NAperGene),
                                                                            np.median(
                                                                                NAperGene),
                                                                            np.min(
                                                                                NAperGene),
                                                                            np.max(NAperGene)))
print("(out of {} tissues)".format(consensusRNAseq_pivot.shape[1]))

print()
print("Number of genes with missing values: {:,}/{:,}".format((NAperGene>0).sum(), len(NAperGene)))

Missing values per gene
------
mean: 0.99 	 median: 0.00 	 min: 0 	 max: 17
(out of 62 tissues)

Number of genes with missing values: 2,441/19,670


___
**Match Ensembl IDs with UniProt IDs**

In [62]:
consensusRNAseq_pivot_2 = consensusRNAseq_pivot.merge(HPAmapping, how="inner", on = "Gene")

glance(consensusRNAseq_pivot_2)

DataFrame: 19,049 rows 	 63 columns


,Gene,B-cells,NK-cells,T-cells,adipose tissue,adrenal gland,amygdala,appendix,basal ganglia,bone marrow,...,testis,thalamus,thymus,thyroid gland,tongue,tonsil,total PBMC,urinary bladder,vagina,uniprotID
0,ENSG00000000003,0.3,0.3,6.6,27.0,9.8,7.0,4.4,6.6,0.1,...,17.5,6.6,2.9,24.6,11.0,19.8,1.4,17.2,11.5,O43657
1,ENSG00000000005,0.0,0.0,0.0,18.4,0.1,0.1,0.4,0.0,0.0,...,0.2,0.0,0.2,0.1,2.6,0.4,0.0,0.2,0.1,Q9H2S6
2,ENSG00000000419,20.7,20.3,25.1,32.3,39.7,20.2,23.3,30.0,32.9,...,25.6,21.7,36.1,34.5,49.1,36.4,15.7,35.3,28.3,O60762
3,ENSG00000000457,8.3,11.8,10.5,10.8,11.4,8.5,15.4,10.6,5.0,...,9.1,8.9,17.2,13.3,9.5,16.0,5.9,16.0,12.7,Q8IZE3
4,ENSG00000000460,2.9,4.9,1.6,3.4,3.1,3.3,7.4,4.9,10.1,...,19.7,5.9,24.6,3.2,2.9,9.5,1.6,4.0,3.3,Q9NSG2


In [63]:
# Sanity check
assert consensusRNAseq_pivot_2.uniprotID.isna().sum() == 0

*Deal with proteins matching several genes*

In [64]:
if len(set(consensusRNAseq_pivot_2.uniprotID)) != len(consensusRNAseq_pivot_2):
    consensusRNAseq_pivot_2 = consensusRNAseq_pivot_2.groupby('uniprotID').max().reset_index()
    print('New number of proteins: {:,}'.format(len(consensusRNAseq_pivot_2)))

New number of proteins: 18,938


In [65]:
# Sanity check
assert len(set(consensusRNAseq_pivot_2.uniprotID)) == len(consensusRNAseq_pivot_2)

---
**Fill missing values with 0**

In [66]:
consensusRNAseq_pivot_3 = consensusRNAseq_pivot_2.fillna(0)
glance(consensusRNAseq_pivot_3)

DataFrame: 18,938 rows 	 63 columns


,uniprotID,Gene,B-cells,NK-cells,T-cells,adipose tissue,adrenal gland,amygdala,appendix,basal ganglia,...,stomach,testis,thalamus,thymus,thyroid gland,tongue,tonsil,total PBMC,urinary bladder,vagina
0,A0A024RBG1,ENSG00000177144,0.0,0.2,0.6,0.5,1.2,0.0,0.0,0.0,...,0.1,1.6,0.0,0.0,1.7,0.0,0.1,0.4,5.8,0.0
1,A0A075B759,ENSG00000271567,1.7,3.4,2.9,0.0,0.4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0
2,A0A075B767,ENSG00000270339,0.4,0.3,1.1,0.0,0.0,0.0,0.0,0.0,...,0.0,0.3,0.0,0.0,0.0,0.0,0.5,0.3,0.0,0.0
3,A0A087WTH1,ENSG00000281991,3.7,6.9,4.3,1.2,0.6,0.3,4.8,0.4,...,6.1,1.4,0.2,2.3,8.7,18.8,6.3,1.7,6.3,2.0
4,A0A087WTH5,ENSG00000276289,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,...,0.0,1.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0


---
**Remove gene ID**

In [67]:
consensusRNAseq_pivot_4 = consensusRNAseq_pivot_3.drop(['Gene'], axis=1)
glance(consensusRNAseq_pivot_4)

DataFrame: 18,938 rows 	 62 columns


,uniprotID,B-cells,NK-cells,T-cells,adipose tissue,adrenal gland,amygdala,appendix,basal ganglia,bone marrow,...,stomach,testis,thalamus,thymus,thyroid gland,tongue,tonsil,total PBMC,urinary bladder,vagina
0,A0A024RBG1,0.0,0.2,0.6,0.5,1.2,0.0,0.0,0.0,5.4,...,0.1,1.6,0.0,0.0,1.7,0.0,0.1,0.4,5.8,0.0
1,A0A075B759,1.7,3.4,2.9,0.0,0.4,0.0,0.0,0.0,2.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0
2,A0A075B767,0.4,0.3,1.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.3,0.0,0.0,0.0,0.0,0.5,0.3,0.0,0.0
3,A0A087WTH1,3.7,6.9,4.3,1.2,0.6,0.3,4.8,0.4,1.2,...,6.1,1.4,0.2,2.3,8.7,18.8,6.3,1.7,6.3,2.0
4,A0A087WTH5,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0


---
**Export**

- v2.0: uses HPA v19.3 and Ensembl v92.38
- v2.1: uses HPA v20.1 and Ensembl v92.38

In [68]:
consensusRNAseq_pivot_export = consensusRNAseq_pivot_4

In [69]:
myVersionHPA = '2-1'

logVersions['HPA']['preprocessed']['consensusRNAseq'] = myVersionHPA

dump_LogVersions(logVersions)

In [70]:
consensusRNAseq_pivot_export.to_pickle(os.path.join(cfg['outputPreprocessingHPA'], "consensusRNAseq_v{}.pkl".format(myVersionHPA)))


In [71]:
foo = pd.read_pickle(os.path.join(cfg['outputPreprocessingHPA'], "consensusRNAseq_v{}.pkl".format('2-0')))

glance(foo)

DataFrame: 18,948 rows 	 63 columns


,uniprotID,B-cells,NK-cells,T-cells,adipose tissue,adrenal gland,amygdala,appendix,basal ganglia,bone marrow,...,substantia nigra,testis,thalamus,thymus,thyroid gland,tongue,tonsil,total PBMC,urinary bladder,vagina
0,A0A024RBG1,0.0,0.2,0.6,0.5,1.2,0.0,0.0,0.0,5.4,...,0.0,1.6,0.0,0.0,1.7,0.0,0.1,0.4,5.8,0.0
1,A0A075B759,1.7,3.4,2.9,0.0,0.4,0.0,0.0,0.0,2.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0
2,A0A075B767,0.4,0.3,1.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.3,0.0,0.0,0.0,0.0,0.5,0.3,0.0,0.0
3,A0A087WTH1,3.7,6.9,4.3,1.2,0.6,0.3,4.8,0.0,1.2,...,0.0,1.4,0.2,2.3,8.7,18.8,6.3,1.7,6.3,2.0
4,A0A087WTH5,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0


In [72]:
foo = pd.read_pickle(os.path.join(cfg['outputPreprocessingHPA'], "consensusRNAseq_v{}.pkl".format('2-1')))

glance(foo)

DataFrame: 18,938 rows 	 62 columns


,uniprotID,B-cells,NK-cells,T-cells,adipose tissue,adrenal gland,amygdala,appendix,basal ganglia,bone marrow,...,stomach,testis,thalamus,thymus,thyroid gland,tongue,tonsil,total PBMC,urinary bladder,vagina
0,A0A024RBG1,0.0,0.2,0.6,0.5,1.2,0.0,0.0,0.0,5.4,...,0.1,1.6,0.0,0.0,1.7,0.0,0.1,0.4,5.8,0.0
1,A0A075B759,1.7,3.4,2.9,0.0,0.4,0.0,0.0,0.0,2.7,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.6,0.0,0.0
2,A0A075B767,0.4,0.3,1.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.3,0.0,0.0,0.0,0.0,0.5,0.3,0.0,0.0
3,A0A087WTH1,3.7,6.9,4.3,1.2,0.6,0.3,4.8,0.4,1.2,...,6.1,1.4,0.2,2.3,8.7,18.8,6.3,1.7,6.3,2.0
4,A0A087WTH5,0.0,0.0,0.1,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.2,0.0,0.0,0.0,0.0,0.0,0.2,0.0,0.0


In [73]:
len(set(foo.uniprotID))

18938

## Subcellular location (IF)

In [74]:
subcellularLocation = pd.read_csv(
    os.path.join(cfg['rawDataHPA'], 'subcellular_location_v{}--{}.tsv'.format(logVersions['HPA']['rawData']['HPA'],logVersions['HPA']['rawData']['Ensembl'])),
    sep = "\t"
)
glance(subcellularLocation)

DataFrame: 12,813 rows 	 14 columns


,Gene,Gene name,Reliability,Main location,Additional location,Extracellular location,Enhanced,Supported,Approved,Uncertain,Single-cell variation intensity,Single-cell variation spatial,Cell cycle dependency,GO id
0,ENSG00000000003,TSPAN6,Approved,Cell Junctions;Cytosol,Nucleoli fibrillar center,NaN,NaN,NaN,Cell Junctions;Cytosol;Nucleoli fibrillar center,NaN,Cytosol,NaN,NaN,Cell Junctions (GO:0030054);Cytosol (GO:000582...
1,ENSG00000000457,SCYL3,Uncertain,Microtubules,Nuclear bodies,NaN,NaN,NaN,NaN,Microtubules;Nuclear bodies,NaN,NaN,NaN,Microtubules (GO:0015630);Nuclear bodies (GO:0...
2,ENSG00000000460,C1orf112,Approved,Mitochondria,NaN,NaN,NaN,NaN,Mitochondria,NaN,NaN,NaN,NaN,Mitochondria (GO:0005739)
3,ENSG00000000938,FGR,Approved,Plasma membrane,Aggresome,NaN,NaN,NaN,Aggresome;Plasma membrane,NaN,NaN,NaN,NaN,Aggresome (GO:0016235);Plasma membrane (GO:000...
4,ENSG00000000971,CFH,Approved,Vesicles,NaN,Predicted to be secreted,NaN,NaN,Vesicles,NaN,NaN,NaN,NaN,Vesicles (GO:0043231)


---
**EDA**

In [75]:
print(subcellularLocation.loc[:,"Reliability"].value_counts())

Approved     6591
Supported    4039
Enhanced     1463
Uncertain     720
Name: Reliability, dtype: int64


In [76]:
print('Number of genes: {:,}'.format(len(set(subcellularLocation.Gene))))

Number of genes: 12,813


___
**Match Ensembl IDs with UniProt IDs**

In [77]:
subcellularLocation_2 = subcellularLocation.merge(HPAmapping, how="inner", on = "Gene")

glance(subcellularLocation_2)

DataFrame: 12,601 rows 	 15 columns


,Gene,Gene name,Reliability,Main location,Additional location,Extracellular location,Enhanced,Supported,Approved,Uncertain,Single-cell variation intensity,Single-cell variation spatial,Cell cycle dependency,GO id,uniprotID
0,ENSG00000000003,TSPAN6,Approved,Cell Junctions;Cytosol,Nucleoli fibrillar center,NaN,NaN,NaN,Cell Junctions;Cytosol;Nucleoli fibrillar center,NaN,Cytosol,NaN,NaN,Cell Junctions (GO:0030054);Cytosol (GO:000582...,O43657
1,ENSG00000000457,SCYL3,Uncertain,Microtubules,Nuclear bodies,NaN,NaN,NaN,NaN,Microtubules;Nuclear bodies,NaN,NaN,NaN,Microtubules (GO:0015630);Nuclear bodies (GO:0...,Q8IZE3
2,ENSG00000000460,C1orf112,Approved,Mitochondria,NaN,NaN,NaN,NaN,Mitochondria,NaN,NaN,NaN,NaN,Mitochondria (GO:0005739),Q9NSG2
3,ENSG00000000938,FGR,Approved,Plasma membrane,Aggresome,NaN,NaN,NaN,Aggresome;Plasma membrane,NaN,NaN,NaN,NaN,Aggresome (GO:0016235);Plasma membrane (GO:000...,P09769
4,ENSG00000000971,CFH,Approved,Vesicles,NaN,Predicted to be secreted,NaN,NaN,Vesicles,NaN,NaN,NaN,NaN,Vesicles (GO:0043231),P08603


In [78]:
# Sanity check
assert subcellularLocation_2.uniprotID.isna().sum() == 0

*Deal with proteins matching several genes*

In [79]:
subcellularLocation_2['GO id'].isna().sum()

0

In [80]:
assert subcellularLocation_2['GO id'].isna().sum() == 0
if len(set(subcellularLocation_2.uniprotID)) != len(subcellularLocation_2):
    subcellularLocation_2 = subcellularLocation_2.groupby('uniprotID').agg({
        'GO id': ';'.join
    }).reset_index()
    print('New number of proteins: {:,}'.format(len(subcellularLocation_2)))

New number of proteins: 12,566


In [81]:
# Sanity check
assert len(set(subcellularLocation_2.uniprotID)) == len(subcellularLocation_2)

---
**Pivot table using bag of words functioon**

In [82]:
bow, vectorizer = createBoW(createGOlist(GOcol = subcellularLocation_2["GO id"], 
                             regex0 = r"(?<=\(GO:)[\d]+(?=\))"))
bow

12566 12566
Shape BoW: (12566, 33)


<12566x33 sparse matrix of type '<class 'numpy.int64'>'
	with 22082 stored elements in Compressed Sparse Row format>

In [83]:
subcellularLocation_BoW = pd.DataFrame(bow.todense())
subcellularLocation_BoW.columns = vectorizer.get_feature_names()
subcellularLocation_BoW['uniprotID'] = subcellularLocation_2['uniprotID']

glance(subcellularLocation_BoW)

DataFrame: 12,566 rows 	 34 columns


,0000776,0001650,0005654,0005694,0005730,0005739,0005764,0005768,0005777,0005783,...,0032154,0034451,0036464,0043231,0045111,0045171,0072686,0090543,1990752,uniprotID
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A0A024RBG1
1,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,A0A087WUL8
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,A0A087WVF3
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,A0A087WXS9
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,A0A087X179


---
**Export**

- v2.0: uses HPA v19.3 and Ensembl v92.38
- v2.1: uses HPA v20.1 and Ensembl v92.38

In [84]:
subcellularLocation_export = subcellularLocation_BoW

In [85]:
myVersionHPA = '2-1'

logVersions['HPA']['preprocessed']['subcellularLocation'] = myVersionHPA

dump_LogVersions(logVersions)

In [86]:
subcellularLocation_export.to_pickle(os.path.join(cfg['outputPreprocessingHPA'], "subcellularLocation_v{}.pkl".format(myVersionHPA)))
